In [11]:
import sys
import os
import numpy as np
# import multiprocessing
import pickle
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
# import time
from scipy.stats import norm
# import datetime as dt

module_path = os.path.abspath(os.path.join('.'))
sys.path.append(module_path + "/../../notebooks/vax_sims/")
from vax_sims_LHS_samples import *
from plot_utils import *
sys.path.append(module_path + "/../../src/simulations_v2")
from stochastic_simulation import StochasticSimulation

In [2]:
UNCERTAINTY_PARAMS = ['vax_susc_mult', 'vax_transmission_mult', 'contacts_per_day_mult', 'outside_infection_rate_mult',
                      'cases_isolated_per_contact_trace', 'initial_ID_prevalence']

UNCERTAINTY_PARAM_RANGES = {
    'vax_susc_mult': (0.097608, 0.941192), # 0.5194 +/- 1.96 * 0.2152
    'vax_transmission_mult': (0.25, 1),
    'contacts_per_day_mult': (0.9,2.7),
    'outside_infection_rate_mult': (1, 5),
    'cases_isolated_per_contact_trace': (0.5,1.5),
    'initial_ID_prevalence': (0.003, 0.0054)
}

def map_lhs_point_to_vax_sim(lhs_point, param_modifiers=None, vax_rates=None, omicron_multiplier=1, social_distance_multiplier=1):
    base_params, base_group_names, contact_matrix, default_vax_rates = load_calibrated_params()
    if vax_rates == None:
        vax_rates = default_vax_rates

    vax_susc_mult = lhs_point[0]
    vax_trans_mult = lhs_point[1]

    contact_matrix = contact_matrix * lhs_point[2] * omicron_multiplier * social_distance_multiplier

    for params in base_params:
        params['daily_outside_infection_p'] = params['daily_outside_infection_p'] * lhs_point[3]
        params['contact_tracing_delay'] = 3


    base_params[0]['cases_isolated_per_contact'] *= lhs_point[4]
    base_params[1]['cases_isolated_per_contact'] *= lhs_point[4]
    base_params[2]['cases_isolated_per_contact'] *= lhs_point[4]
    
    base_params[0]['initial_ID_prevalence'] = lhs_point[5]
    base_params[1]['initial_ID_prevalence'] = lhs_point[5]
    base_params[2]['initial_ID_prevalence'] = lhs_point[5]
    
    vax_sim = generate_vax_unvax_multigroup_sim(base_params, base_group_names,
                                    vax_rates, contact_matrix,
                                    vax_trans_mult, vax_susc_mult)

    update_vax_sim_params(vax_sim, param_modifiers)

    return vax_sim

def load_posterior_df():
    df = pd.read_csv('../vax_sims/posterior_csvs/21_10_08_14:44_posteriors.csv')
    return df

df = load_posterior_df()

In [3]:
df.sort_values('posterior', ascending=False).head(1)[UNCERTAINTY_PARAMS].iloc[0]

vax_susc_mult                       0.551899
vax_transmission_mult               0.495139
contacts_per_day_mult               1.447773
outside_infection_rate_mult         3.710611
cases_isolated_per_contact_trace    0.914374
initial_ID_prevalence               0.005532
Name: 2096, dtype: float64

In [4]:
MLE_point = list(df.sort_values('posterior', ascending=False).head(1)[UNCERTAINTY_PARAMS].iloc[0])
MLE_point

[0.5518990161488261,
 0.4951388585161961,
 1.4477728451100855,
 3.710610670072047,
 0.9143735965091287,
 0.00553175759432151]

In [8]:
group_params = load_params("../vax_sims/vax_sim_nominal_params/ug_greek_athlete_nominal.yaml")[1]

vax_susc_mult = MLE_point[0]
vax_trans_mult = MLE_point[1]

# Contacts per day should be 7.59 (7.36 + 0.23) from the contact matrix
omicron_multiplier = 5
social_distance_multiplier = 1
group_params['expected_contacts_per_day'] = 7.59 * vax_susc_mult * vax_trans_mult * MLE_point[2] * omicron_multiplier * social_distance_multiplier

group_params['daily_outside_infection_p'] = group_params['daily_outside_infection_p'] * MLE_point[3] * 10
group_params['cases_isolated_per_contact'] *= MLE_point[4]
group_params['initial_ID_prevalence'] = 0.04

# 66% of (3533+8434) ~= 8018
group_params['population_size'] = 8018
group_params['test_population_fraction'] = 1/7

In [9]:
sim = StochasticSimulation(group_params)

In [12]:
sim_results = list()
for _ in range(100):
    sim_results.append(sim.run_new_trajectory(35))

In [15]:
pickle.dump(sim_results, open('UG_virtual_sim_results.p', 'wb'))

In [13]:
sim_results[0]

,S,QS,QI,R,E_0,E_1,E_2,E_3,E_4,E_5,...,SyID_severe_17,SyID_severe_18,SyID_severe_19,cumulative_mild,cumulative_severe,cumulative_outside_infections,severity_0,severity_1,severity_2,severity_3
0,7697,0,0,0,0,0,0,0,0,0,...,0,0,2,10,13,0,10,12.6803,0.234206,0.0855406
1,7168,0,35,1,152,134,103,49,23,6,...,0,0,0,49,53,1,49,51.6964,0.954838,0.348743
2,6640,0,97,10,286,244,132,54,20,6,...,6,1,1,121,118,3,121,115.098,2.12587,0.776446
3,5843,0,169,34,445,342,202,70,36,3,...,4,2,0,205,229,13,205,223.368,4.12562,1.50683
4,4725,0,299,57,638,525,262,85,37,15,...,2,0,2,336,358,15,336,349.195,6.44966,2.35566
5,3208,0,542,92,922,684,296,146,51,27,...,4,3,2,568,601,17,568,586.218,10.8275,3.95461
6,1634,0,915,170,1107,720,315,129,69,12,...,7,0,3,891,1000,17,891,975.404,18.0158,6.58005
7,265,0,1457,293,1088,599,262,131,46,12,...,7,7,8,1373,1528,18,1373,1490.42,27.5282,10.0543
8,0,0,2232,471,662,95,54,23,7,3,...,13,13,5,2026,2254,18,2026,2198.56,40.6076,14.8314
9,0,0,3237,731,0,0,0,0,0,0,...,15,6,7,2826,3104,18,2826,3027.65,55.9211,20.4245
